# Creating Database using SQL Lite

In [44]:
# Loading SQL extention
%load_ext sql

# Creating air_quality_armenia.db file
%sql sqlite:///air_quality_armenia.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


## Files description

sensors.csv - Contains the list of sensors and their stations. Updated hourly.

    id - unique sensor ID
    station_id - station ID, several sensors can be located at the same station
    city_slug - city of the sensor
    district_slug - district of the sensor, can be empty
    address_en - address in English
    title - title of the sensor, can be empty
    provider - sensor provider: sensor_community, yerevan_gis, purple_air, clarity.
    external_id - sensor ID in the provider's system
    sensor_type - sensor type: SDS011, BME280 or other
    latitude
    longitude
    altitude - altitude in meters, can be empty
    first_measurement_time - timestamp of the first measurement
    last_measurement_time - timestamp of the last measurement
    is_suspicious - true if the sensor data looks suspicious, false otherwise

sensor_avg_daily.csv - Contains the daily average values of the sensors. Updated daily.

    sensor_id - sensor ID
    timestamp - date of the measurement in the format YYYY-MM-DD 00:00:00
    avg_pm2.5 - average corrected PM2.5 value
    avg_pm10 - average PM10 value
    avg_temperature - average temperature in Celsius
    avg_pressure - average pressure in Pa
    avg_humidity - average humidity in percent

station_avg_daily.csv - Contains the daily average values of the stations. Updated daily.

    station_id - station ID
    timestamp - date of the measurement in the format YYYY-MM-DD 00:00:00
    avg_pm2.5 - average corrected PM2.5 value
    avg_pm10 - average PM10 value
    avg_temperature - average temperature in Celsius
    avg_pressure - average pressure in Pa
    avg_humidity - average humidity in percent

sensor_avg_hourly/sensor_avg_hourly_{year}.csv
 - Contains the hourly average values of the sensors. Updated daily. Last file updated hourly.
   Same format as sensor_avg_daily.csv.

station_avg_hourly/station_avg_hourly_{year}.csv
 - Contains the hourly average values of the stations. Updated daily. Last file updated hourly.
   Same format as station_avg_daily.csv.

measurements/measurements_{year}_{month}.csv
 - Contains the raw measurements of the sensors. Updated daily. Last file updated hourly.

    sensor_id - sensor ID
    timestamp - date of the measurement in the format YYYY-MM-DD HH:MM:SS
    time_start - start of the measurement period in the format YYYY-MM-DD HH:MM:SS (for average values)
    time_end - end of the measurement period in the format YYYY-MM-DD HH:MM:SS (for average values)
    pm2.5 - raw PM2.5 value
    pm2.5_corrected - corrected PM2.5 value
    pm10 - raw PM10 value
    temperature - temperature in Celsius
    pressure - pressure in Pa
    humidity - humidity in percent

## Creating database from csv files using SQL Lite

In [45]:
import pandas as pd
import sqlite3

# Create connection to SQLite database
conn = sqlite3.connect('air_quality_armenia.db')

try:
    # Load sensors.csv
    sensors_df = pd.read_csv('sensors.csv')
    sensors_df.to_sql('sensors', conn, if_exists='replace', index=False)
    
    # Load sensor_avg_daily.csv
    sensor_daily_df = pd.read_csv('sensor_avg_daily.csv')
    sensor_daily_df.to_sql('sensor_avg_daily', conn, if_exists='replace', index=False)
    
    # Load station_avg_daily.csv
    station_daily_df = pd.read_csv('station_avg_daily.csv')
    station_daily_df.to_sql('station_avg_daily', conn, if_exists='replace', index=False)
    
    # Load hourly data (example for 2024)
    sensor_hourly_df = pd.read_csv('sensor_avg_hourly_2024.csv')
    sensor_hourly_df.to_sql('sensor_avg_hourly_2024', conn, if_exists='replace', index=False)
    
    station_hourly_df = pd.read_csv('station_avg_hourly_2024.csv')
    station_hourly_df.to_sql('station_avg_hourly_2024', conn, if_exists='replace', index=False)

    print("Database created and data loaded successfully!")
finally:
    # Close the connection
    conn.close()

Database created and data loaded successfully!


In [57]:
# Connect to the database
conn = sqlite3.connect('air_quality_armenia.db')

query = """
SELECT strftime('%Y-%m-%d', timestamp) AS day
     , city_slug as city
     , AVG("pm2.5") AS "avg_pm2.5_daily"
FROM sensor_avg_hourly_2024
    JOIN sensors on sensor_avg_hourly_2024.sensor_id = sensors.id
WHERE city_slug = 'yerevan'
  AND is_suspicious = 0
GROUP BY day, city
ORDER BY day, city
"""
armenia_2024_daily = pd.read_sql_query(query, conn)

conn.close()

armenia_2024_daily.head()

,day,city,avg_pm2.5_daily
0,2024-01-01,yerevan,57.438624
1,2024-01-02,yerevan,54.354692
2,2024-01-03,yerevan,56.676744
3,2024-01-04,yerevan,39.712145
4,2024-01-05,yerevan,29.708010
